In [2]:
import pandas as pd

In [3]:
nur = pd.read_csv("National Universities Rankings.csv", index_col=0)
nur.shape

(231, 7)

ROUGH OUTLINE
1. extract the state short code from location
2. extract the year founded from description
3. clean up the fees, in-state and enrollment columns
4. clean up the columns names

* 51 states

5. seperate each state schools
6. each state will be a table of it's own in the database for database optimization
7. create a state table
8. can rank school inside each state based on enrollment

-- EDA

9. Top school based on rank, tuition fees, instate fees, and enrollment
10. Top 2 schools within 56 states
11. oldest schools based on year founded
12. oldest school within 51 states

-- App interface

13. A brief overview with visuals from top school based on rank (overall)
14. selection box to select state. Once selected, brief overview with a visual of the top school in that state.
* Overview will include details based on the available data like average tuition and top ranked school.
15. Another section where we take in input from the user to recommend a movie within a choosen state.
16. If no state is choosen, we recommend based on the user location, and closest state.
* For closest state, we can do some resear to know which state is closer to each other (Feature engineering)  


In [4]:
nur.sample(20)

,Name,Location,Rank,Description,Tuition and fees,In-state,Undergrad Enrollment
index,,,,,,,
11,California Institute of Technology,"Pasadena, CA",12,"Caltech, which focuses on science and engineer...","$47,577",NaN,"1,001"
164,Maryville University of St. Louis,"St Louis, MO",164,"Founded in 1872, Maryville University of St. L...","$27,958",NaN,"2,795"
157,University of Massachusetts--Lowell,"Lowell, MA",152,"Founded in 1894, University of Massachusetts--...","$29,125","$13,427","13,266"
206,University of North Carolina--Charlotte,"Charlotte, NC",202,"Founded in 1946, University of North Carolina-...","$20,193","$6,617","22,732"
219,Benedictine University,"Lisle, IL",220,"Founded in 1887, Benedictine University is a p...","$32,170",NaN,"3,347"
5,Stanford University,"Stanford, CA",5,Stanford University's pristine campus is locat...,"$47,940",NaN,"6,999"
121,University of Kansas,"Lawrence, KS",118,"Founded in 1865, University of Kansas is a pub...","$25,932","$10,549","19,245"
0,Princeton University,"Princeton, NJ",1,"Princeton, the fourth-oldest college in the Un...","$45,320",NaN,"5,402"
123,The Catholic University of America,"Washington, DC",124,"Catholic University of America, as its name su...","$42,536",NaN,"3,480"


In [5]:
nur['Location'].apply(lambda x: x.split(',')[1]).unique()

array([' NJ', ' MA', ' IL', ' CT', ' NY', ' CA', ' NC', ' PA', ' MD',
       ' NH', ' RI', ' TX', ' IN', ' TN', ' MO', ' GA', ' DC', ' VA',
       ' MI', ' OH', ' LA', ' FL', ' WI', ' WA', ' SC', ' UT', ' MN',
       ' DE', ' CO', ' IA', ' OK', ' VT', ' AL', ' OR', ' NE', ' KS',
       ' AZ', ' KY', ' AR', ' MS', ' HI', ' ID', ' WY', ' NM', ' ME',
       ' WV', ' ND', ' NV', ' SD', ' AK', ' MT'], dtype=object)

In [6]:
len(nur['Location'].apply(lambda x: x.split(',')[1]).unique())

51

In [75]:
# # Get the unique values in each column
# unique_values = {}
# for column in nur.columns:
#     unique_values[column] = nur[column].unique()

# # Print the unique values
# for column, values in unique_values.items():
#     print(f"Unique values in column '{column}':")
#     print(values)
#     print()


In [8]:
df = nur.copy()

In [9]:
df.head()

,Name,Location,Rank,Description,Tuition and fees,In-state,Undergrad Enrollment
index,,,,,,,
0,Princeton University,"Princeton, NJ",1,"Princeton, the fourth-oldest college in the Un...","$45,320",NaN,"5,402"
1,Harvard University,"Cambridge, MA",2,"Harvard is located in Cambridge, Massachusetts...","$47,074",NaN,"6,699"
2,University of Chicago,"Chicago, IL",3,"The University of Chicago, situated in Chicago...","$52,491",NaN,"5,844"
3,Yale University,"New Haven, CT",3,"Yale University, located in New Haven, Connect...","$49,480",NaN,"5,532"
4,Columbia University,"New York, NY",5,"Columbia University, located in Manhattan's Mo...","$55,056",NaN,"6,102"


In [10]:
df.nunique()

Name                    231
Location                192
Rank                     63
Description             231
Tuition and fees        230
In-state                132
Undergrad Enrollment    230
dtype: int64

### Data Cleaning

In [11]:
df.isnull().sum(axis=0)

Name                     0
Location                 0
Rank                     0
Description              0
Tuition and fees         0
In-state                98
Undergrad Enrollment     0
dtype: int64

The in-state column shows the tuition fee for in-sate students therefore we replace missing values here with the mean fee.

In [12]:
df["In-state"].unique()

array([nan, '$13,509 ', '$12,836 ', '$15,722 ', '$13,856 ', '$8,834 ',
       '$18,687 ', '$12,212 ', '$14,022 ', '$14,750 ', '$14,046 ',
       '$14,705 ', '$15,698 ', '$10,488 ', '$6,389 ', '$17,900 ',
       '$10,037 ', '$10,753 ', '$11,634 ', '$9,806 ', '$10,002 ',
       '$14,066 ', '$10,181 ', '$14,240 ', '$18,618 ', '$14,372 ',
       '$13,790 ', '$10,176 ', '$12,852 ', '$14,288 ', '$13,559 ',
       '$12,520 ', '$17,383 ', '$14,070 ', '$8,325 ', '$9,044 ',
       '$10,388 ', '$9,507 ', '$8,880 ', '$11,531 ', '$17,300 ',
       '$9,026 ', '$10,696 ', '$7,770 ', '$9,770 ', '$10,470 ',
       '$10,762 ', '$12,668 ', '$17,624 ', '$11,854 ', '$7,969 ',
       '$9,518 ', '$8,628 ', '$8,065 ', '$8,518 ', '$14,634 ', '$15,688 ',
       '$13,581 ', '$10,549 ', '$10,872 ', '$10,158 ', '$11,080 ',
       '$11,484 ', '$9,350 ', '$9,842 ', '$16,108 ', '$13,829 ',
       '$8,820 ', '$11,000 ', '$7,744 ', '$11,300 ', '$10,366 ',
       '$11,041 ', '$11,744 ', '$7,084 ', '$9,124 ', '$12,162 ',

In [13]:
df.dtypes

Name                    object
Location                object
Rank                     int64
Description             object
Tuition and fees        object
In-state                object
Undergrad Enrollment    object
dtype: object

The "Tuition and fees" and "In-state" features are of the wrong data types and carry symbols, 
* first replace $ signs and commas 
* Put 0 as placeholder for NaN values
* Convert to integer dtype
* replace 0's with mean

In [14]:
df.sample(5)

,Name,Location,Rank,Description,Tuition and fees,In-state,Undergrad Enrollment
index,,,,,,,
200,University of Nevada--Reno,"Reno, NV",197,The University of Nevada--Reno is located on t...,"$21,052","$7,142","17,770"
47,University of Miami,"Coral Gables, FL",44,"Located in Southern Florida, the University of...","$47,004",NaN,"11,122"
53,Ohio State University--Columbus,"Columbus, OH",54,"Located in the state capital of Columbus, The ...","$29,229","$10,037","45,289"
186,Widener University,"Chester, PA",183,"Widener University is located in Chester, Penn...","$42,870",NaN,"3,554"
32,University of Rochester,"Rochester, NY",32,The University of Rochester describes itself a...,"$50,142",NaN,"6,304"


In [15]:
# Remove the dollar sign ($) and commas
df['Tuition and fees'] = df['Tuition and fees'].str.strip('$')
df['In-state'] = df['In-state'].str.strip('$')


df["Undergrad Enrollment"] = df["Undergrad Enrollment"].str.replace(',', '')
df['Tuition and fees'] = df['Tuition and fees'].str.replace(',', '')
df['In-state'] = df['In-state'].str.replace(',', '')

In [16]:
df.head()

,Name,Location,Rank,Description,Tuition and fees,In-state,Undergrad Enrollment
index,,,,,,,
0,Princeton University,"Princeton, NJ",1,"Princeton, the fourth-oldest college in the Un...",45320,NaN,5402
1,Harvard University,"Cambridge, MA",2,"Harvard is located in Cambridge, Massachusetts...",47074,NaN,6699
2,University of Chicago,"Chicago, IL",3,"The University of Chicago, situated in Chicago...",52491,NaN,5844
3,Yale University,"New Haven, CT",3,"Yale University, located in New Haven, Connect...",49480,NaN,5532
4,Columbia University,"New York, NY",5,"Columbia University, located in Manhattan's Mo...",55056,NaN,6102


In [17]:
df.dtypes

Name                    object
Location                object
Rank                     int64
Description             object
Tuition and fees        object
In-state                object
Undergrad Enrollment    object
dtype: object

In [18]:
# convert the column datatype to integer
df['In-state'] = df['In-state'].fillna('0').astype(int)
df["Undergrad Enrollment"] = df["Undergrad Enrollment"].astype(int)

In [19]:
mean_instate = int(df["In-state"].mean())
mean_instate

6273

In [20]:
df['In-state'] = df['In-state'].replace(0, mean_instate)
df['In-state'].isna().sum()

0

In [21]:
df["In-state"].unique()

array([ 6273, 13509, 12836, 15722, 13856,  8834, 18687, 12212, 14022,
       14750, 14046, 14705, 15698, 10488,  6389, 17900, 10037, 10753,
       11634,  9806, 10002, 14066, 10181, 14240, 18618, 14372, 13790,
       10176, 12852, 14288, 13559, 12520, 17383, 14070,  8325,  9044,
       10388,  9507,  8880, 11531, 17300,  9026, 10696,  7770,  9770,
       10470, 10762, 12668, 17624, 11854,  7969,  9518,  8628,  8065,
        8518, 14634, 15688, 13581, 10549, 10872, 10158, 11080, 11484,
        9350,  9842, 16108, 13829,  8820, 11000,  7744, 11300, 10366,
       11041, 11744,  7084,  9124, 12162, 13296,  7778, 13262, 13670,
       13427,  7766, 11264, 12862,  6410,  9628, 13076, 11732,  7232,
       10738,  5055,  9654,  7502, 11771, 10622,  6368,  7071, 10628,
        7632, 10012,  9046, 10796, 10710, 11029,  9205,  9128, 10404,
        7142,  6436, 12150,  8853,  8172,  7799,  6617,  4965,  8457,
        6946,  7031, 10048,  9551, 14334, 13481,  6446,  6313,  6729,
       16311, 13435,

In [22]:
# Confirming that only 98 values were replaced with the mean
df[df["In-state"]==6273].count()

Name                    98
Location                98
Rank                    98
Description             98
Tuition and fees        98
In-state                98
Undergrad Enrollment    98
dtype: int64

In [23]:
df.dtypes

Name                    object
Location                object
Rank                     int64
Description             object
Tuition and fees        object
In-state                 int64
Undergrad Enrollment     int64
dtype: object

In [24]:
df["Tuition and fees"] = df["Tuition and fees"].astype(int)
df["Tuition and fees"].dtype

dtype('int64')

In [25]:
df.sample(5)

,Name,Location,Rank,Description,Tuition and fees,In-state,Undergrad Enrollment
index,,,,,,,
93,University of Colorado--Boulder,"Boulder, CO",92,"The University of Colorado Boulder, called CUŒ...",35079,11531,27010
84,University of Iowa,"Iowa City, IA",82,The University of Iowa offers top-notch academ...,28413,8325,23357
141,University of Mississippi,"University, MS",135,"The University of Mississippi, better known as...",22012,7744,18785
83,Texas Christian University,"Fort Worth, TX",82,Texas Christian University is the biggest reli...,42670,6273,8894
200,University of Nevada--Reno,"Reno, NV",197,The University of Nevada--Reno is located on t...,21052,7142,17770


### Feature Engineering

In [26]:
# Split string column into two new columns
df[['State', 'State Code']] = df.Location.str.split(",", expand = True)
df.head()

,Name,Location,Rank,Description,Tuition and fees,In-state,Undergrad Enrollment,State,State Code
index,,,,,,,,,
0,Princeton University,"Princeton, NJ",1,"Princeton, the fourth-oldest college in the Un...",45320,6273,5402,Princeton,NJ
1,Harvard University,"Cambridge, MA",2,"Harvard is located in Cambridge, Massachusetts...",47074,6273,6699,Cambridge,MA
2,University of Chicago,"Chicago, IL",3,"The University of Chicago, situated in Chicago...",52491,6273,5844,Chicago,IL
3,Yale University,"New Haven, CT",3,"Yale University, located in New Haven, Connect...",49480,6273,5532,New Haven,CT
4,Columbia University,"New York, NY",5,"Columbia University, located in Manhattan's Mo...",55056,6273,6102,New York,NY


In [28]:
df["State Code"].nunique()

51

In [33]:
df["State Code"]=df["State Code"].str.strip(" ")

In [62]:
# Viewing the features of schools per state
nj = df[df["State Code"]=="NJ"]
nj_tuition = pd.DataFrame(nj.groupby("Name").agg({"Tuition and fees":"mean"}).sort_values("Tuition and fees",ascending=False))
nj_instate = pd.DataFrame(nj.groupby("Name").agg({"In-state":"mean"}).sort_values("In-state",ascending=False))
nj_enroll = pd.DataFrame(nj.groupby("Name").agg({"Undergrad Enrollment":"mean"}).sort_values("Undergrad Enrollment",ascending=False))
print(nj_tuition)
print("----------------------------------------------")
print(nj_instate)
print("----------------------------------------------")
print(nj_enroll)

                                    Tuition and fees
Name                                                
Stevens Institute of Technology              48838.0
Princeton University                         45320.0
Seton Hall University                        39258.0
New Jersey Institute of Technology           30326.0
Rutgers University--New Brunswick            30023.0
Rutgers University--Newark                   29480.0
Montclair State University                   20318.0
----------------------------------------------
                                    In-state
Name                                        
New Jersey Institute of Technology   16108.0
Rutgers University--New Brunswick    14372.0
Rutgers University--Newark           13829.0
Montclair State University           11771.0
Princeton University                  6273.0
Seton Hall University                 6273.0
Stevens Institute of Technology       6273.0
----------------------------------------------
                        

In [60]:
a.head(2)

,Tuition and fees
Name,
Stevens Institute of Technology,48838.0
Princeton University,45320.0


In [72]:
# function to view feature reports of schools in each state
def show_report(state_code):
    state_df = df[df["State Code"]==state_code]
    tuition = pd.DataFrame(state_df.groupby("Name").agg({"Tuition and fees":"mean"}).sort_values("Tuition and fees",ascending=False))
    instate = pd.DataFrame(state_df.groupby("Name").agg({"In-state":"mean"}).sort_values("In-state",ascending=False))
    enroll = pd.DataFrame(state_df.groupby("Name").agg({"Undergrad Enrollment":"mean"}).sort_values("Undergrad Enrollment",ascending=False))
    print(tuition)
    print("-------------------------------------------------------")
    print(instate)
    print("-------------------------------------------------------")
    print(enroll)
    

In [73]:
show_report("CT")

                           Tuition and fees
Name                                       
Yale University                     49480.0
University of Hartford              37790.0
University of Connecticut           35858.0
-------------------------------------------------------
                           In-state
Name                               
University of Connecticut   14066.0
University of Hartford       6273.0
Yale University              6273.0
-------------------------------------------------------
                           Undergrad Enrollment
Name                                           
University of Connecticut               18826.0
Yale University                          5532.0
University of Hartford                   5246.0


In [74]:
show_report("IL")

                                          Tuition and fees
Name                                                      
University of Chicago                              52491.0
Northwestern University                            50855.0
Illinois Institute of Technology                   45214.0
Loyola University Chicago                          41384.0
DePaul University                                  37626.0
Benedictine University                             32170.0
University of Illinois--Urbana-Champaign           31320.0
Southern Illinois University--Carbondale           27130.0
University of Illinois--Chicago                    26526.0
Northern Illinois University                       23799.0
Illinois State University                          20886.0
-------------------------------------------------------
                                          In-state
Name                                              
University of Illinois--Urbana-Champaign   15698.0
Northern Illinois Univer